In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import itertools
import numpy as np
import torch

In [2]:
def load_data(path):
    data = pd.read_csv(path, sep="\t")
    return data
    
path = '../datasets/subreddit-worldnews-DJIA-dataset/combined-rworldnews-DJIA.txt'
raw_data = load_data(path)
print(raw_data.head(5))

         Date  Label                                               News
0  2008-03-03  False  War brewing as Colombia raids Ecuador, kills F...
1  2008-03-04  False  The Gaza Bombshell: "Vanity Fair" reports on h...
2  2008-03-05   True  Colombian President to bring genocide charges ...
3  2008-03-06  False  Do Colombia and the US want war in South Ameri...
4  2008-03-07  False  A gay teenager who sought sanctuary in Britain...


In [3]:
def clean_data(raw_data):
    all_news = {}
    for index, row in raw_data.iterrows():
        news = row['News']
        date = row['Date']
        headlines = news.split("<.>")
        daily_headlines = []
        for headline in headlines:
            headline = headline.strip('"b')
            #converting the sentences into words
            tokens = word_tokenize(headline)
            #converting to lowercas
            tokens = [w.lower() for w in tokens]
            #remvoing non english alphabetic character
            words = [word for word in tokens if word.isalpha()]
            #removing the stop words
            #words = [w for w in words if not w in stop_words]
            #print(words)
            daily_headlines.append(words)
        all_news[date] = daily_headlines
    return all_news


cleaned_data = clean_data(raw_data)
print(len(cleaned_data))
for date in cleaned_data:
    print(cleaned_data[date])
    break

3003
[['war', 'brewing', 'as', 'colombia', 'raids', 'ecuador', 'kills', 'farc', 'in', 'command', 'due', 'to', 'east', 'nature', 'of', 'event', 'world', 'largely', 'fails', 'to', 'care'], ['you', 'causing', 'a', 'war', 'reddit'], ['do', 'you', 'realize', 'stories', 'on', 'are', 'real'], ['medvedev', 'wins', 'by', 'big', 'margin', 'in', 'russia'], ['iran', 'send', 'iraq', 'a', 'bebo', 'friend', 'request', 'iraq', 'accepts', 'moves', 'iran', 'into', 'top', 'then', 'watches', 'amy', 'winehouse', 'video', 'bush', 'still', 'on', 'myspace', 'remains', 'blissfully', 'unaware'], ['us', 'christens', 'steel', 'warship'], ['hiv', 'rates', 'will', 'rise', 'by', 'up', 'to', 'per', 'cent', 'over', 'the', 'next', 'seven', 'years', 'partly', 'because', 'fewer', 'people', 'are', 'using', 'condoms'], ['whaling', 'ship', 'attacked', 'by', 'activists'], ['israel', 'bombs', 'hamas', 'prime', 'minister', 'office', 'in', 'gaza'], ['oscar', 'winning', 'french', 'actress', 'questions', 'fairy', 'tale'], ['early

In [4]:
def dictionary(cleaned_data,threshold):
    caps = []
    for date in cleaned_data:
        for news in cleaned_data[date]:
            caps.append(news)

    word_freq = nltk.FreqDist(itertools.chain(*caps))
    id_to_word = ['<pad>'] + [word for word, cnt in word_freq.items() if cnt >= threshold] + ['<unk>']
    word_to_id = {word:idx for idx, word in enumerate(id_to_word)}
    
    return id_to_word, word_to_id

id_to_word, word_to_id = dictionary(cleaned_data, 5)
print(word_to_id)

{'<pad>': 0, 'war': 1, 'brewing': 2, 'as': 3, 'colombia': 4, 'raids': 5, 'ecuador': 6, 'kills': 7, 'farc': 8, 'in': 9, 'command': 10, 'due': 11, 'to': 12, 'east': 13, 'nature': 14, 'of': 15, 'event': 16, 'world': 17, 'largely': 18, 'fails': 19, 'care': 20, 'you': 21, 'causing': 22, 'a': 23, 'reddit': 24, 'do': 25, 'realize': 26, 'stories': 27, 'on': 28, 'are': 29, 'real': 30, 'medvedev': 31, 'wins': 32, 'by': 33, 'big': 34, 'margin': 35, 'russia': 36, 'iran': 37, 'send': 38, 'iraq': 39, 'friend': 40, 'request': 41, 'accepts': 42, 'moves': 43, 'into': 44, 'top': 45, 'then': 46, 'watches': 47, 'amy': 48, 'video': 49, 'bush': 50, 'still': 51, 'remains': 52, 'unaware': 53, 'us': 54, 'steel': 55, 'warship': 56, 'hiv': 57, 'rates': 58, 'will': 59, 'rise': 60, 'up': 61, 'per': 62, 'cent': 63, 'over': 64, 'the': 65, 'next': 66, 'seven': 67, 'years': 68, 'partly': 69, 'because': 70, 'fewer': 71, 'people': 72, 'using': 73, 'condoms': 74, 'whaling': 75, 'ship': 76, 'attacked': 77, 'activists': 78

In [23]:
def tokenization(cleaned_data, word_to_id):
    tokens, dates = [], []
    lengths = []
    for date in cleaned_data:
        daily_headlines = []
        for headline in cleaned_data[date]:
            token = []
            for word in headline:
                if word in word_to_id:
                    token.append(word_to_id[word])
                else:
                    token.append(word_to_id['<unk>'])
            lengths.append(len(token))
            daily_headlines.append(token)
        tokens.append(daily_headlines)
        dates.append(date)
    #tokens = np.array(tokens).astype('int32')
    dates = np.array(dates)
    
    return tokens, dates, lengths

tokens, dates, lengths = tokenization(cleaned_data, word_to_id)
print(len(lengths))
print(len(tokens))
print(len(tokens[0]))
#print(tokens[0])

90722
3003
34


In [34]:
lengths = []
for date in cleaned_data:
    daily_lengths = []
    for headline in cleaned_data[date]:
        daily_lengths.append(len(headline))
    lengths.append(daily_lengths)
print(len(lengths))
print(lengths[0])

3003
[21, 5, 7, 7, 25, 4, 21, 5, 8, 7, 18, 7, 13, 6, 10, 8, 11, 8, 12, 6, 18, 7, 7, 9, 19, 6, 18, 7, 8, 7, 12, 4, 8, 7]


In [20]:
i = 0
flat_tokens = []
for token in tokens:
    i += 1
    flat_token = []
    for t in token[0:10]:
        for word in t:
            flat_token.append(word)
    flat_tokens.append(flat_token)

max_len = 100  
min_len = 1000
for flat_token in flat_tokens:
    if len(flat_token) > max_len:
        max_len = len(flat_token)
    if len(flat_token) < min_len:
        min_len = len(flat_token)
print(max_len)
print(min_len)

334
79


In [37]:
import torch.nn as nn
import torch

pre_emb = torch.ones((10, 5))
print(pre_emb.shape)
print(pre_emb)
# an Embedding module containing 10 tensors of size 3
#embedding_sum = nn.EmbeddingBag(10, 3, mode='sum')
embedding_sum = nn.EmbeddingBag.from_pretrained(pre_emb, mode='sum')
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(input.shape)
offsets = torch.LongTensor([0, 5])
embedding_sum(input, offsets)

torch.Size([10, 5])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
torch.Size([10])


tensor([[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]])

In [36]:
tensor = torch.tensor([[1, 2],[3, 4]])
offsets = [0]
for t in tensor:
    offsets.append(t.size(0))
offsets = offsets[:-1]
offsets = torch.tensor(offsets).cumsum(0)
print(offsets)

tensor([0, 2])
